# 运维内容

# tmux

## 1 常规的tmux 操作

[tmux 常规操作文章](https://www.ruanyifeng.com/blog/2019/10/tmux.html)

## 2 使用shell 脚本控制tmux

/opt/homebrew/bin/tmux new-session -d -s jupyter_window2 'source /Users/zhaoxuefeng/opt/miniconda3/bin/activate data_server_web && python -m data_server_web.server --server_port 8701'

In [ ]:

source /Users/zhaoxuefeng/opt/miniconda3/bin/activate py310_new  # 执行会带有环境

/opt/homebrew/bin/tmux new-session -d -s jupyter_11 'jupyter lab --NotebookApp.notebook_dir=/Users/zhaoxuefeng/Documents/Project'

/opt/homebrew/bin/tmux new-session -d -s jupyter_window2 'source /Users/zhaoxuefeng/opt/miniconda3/bin/activate data_server_web && python -m data_server_web.server --server_port 8700'

In [ ]:
#!/bin/bash

# Create a new tmux session
/opt/homebrew/bin/tmux new-session -d -s jupyter_window2

# Send commands to the tmux session
/opt/homebrew/bin/tmux send-keys -t jupyter_window2 "source /path/to/conda/bin/activate data_server_web" C-m
/opt/homebrew/bin/tmux send-keys -t jupyter_window2 "python -m data_server_web.server --server_port 8700" C-m

# Optional: attach to the tmux session if you want to monitor it
#/opt/homebrew/bin/tmux attach-session -t jupyter_window2


## 3开机自动拉起

In [ ]:
# !vim ~/Library/LaunchAgents/com.user.tmux.plist
添加以下内容
# 脚本一定不要写在documents 中

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <key>Label</key>
    <string>com.user.tmux</string>
    <key>ProgramArguments</key>
    <array>
        <string>/bin/bash</string>
        <string>/Users/zhaoxuefeng/start_tmux_jupyter.sh</string>
    </array>
    <key>RunAtLoad</key>
    <true/>
    <!-- Remove KeepAlive to avoid constant restarting -->
    <!-- <key>KeepAlive</key>
    <true/> -->
</dict>
</plist>

In [ ]:
launchctl unload ~/Library/LaunchAgents/com.user.tmux.plist
launchctl load ~/Library/LaunchAgents/com.user.tmux.plist

# Docker 

## download_miniconda.sh

In [4]:
# %load download_miniconda.sh
#!/bin/bash
# 检测系统架构
ARCH=$(uname -m)
MINICONDA_URL=""
if [ "$ARCH" = "x86_64" ]; then
    # 如果是x86_64架构
    MINICONDA_URL="https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh"
elif [ "$ARCH" = "aarch64" ]; then
    # 如果是ARM架构
    MINICONDA_URL="https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh"
else
    echo "不支持的架构: $ARCH"
    exit 1
fi
# 下载Miniconda安装脚本
wget $MINICONDA_URL -O miniconda.sh



## Dockerfile

In [3]:
# %load Dockerfile2
# 设置基础镜像
#         本地源地址更换 替换或更新 pip310_install  和pip_install 文件
#         机器变更 只需要调整  registerd.json host_config.json 的内容就行
#         账号密码管理 暂无
FROM ubuntu:latest
COPY download_miniconda.sh /download_miniconda.sh
SHELL ["/bin/bash", "-c"]
RUN apt-get update \
     && DEBIAN_FRONTEND=noninteractive apt-get install libglib2.0-0 -y \
	 && apt-get install -y build-essential wget vim gcc rsync sshpass git libgl1-mesa-glx 
RUN echo "下载并安装miniconda" \
	&& chmod +x /download_miniconda.sh \
	&& bash /download_miniconda.sh \
    && bash miniconda.sh -b -p /opt/conda \
    && rm -rf /download_miniconda.sh \
    && rm -rf miniconda.sh \
	&& ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh
RUN echo "启动conda 修改配置" \
	&& echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && source /root/.bashrc \
	&& source /opt/conda/bin/activate && echo "source /opt/conda/bin/activate" >> ~/.bashrc \
	&& conda create -n py310 python=3.10 -y \
	&& /opt/conda/envs/py310/bin/pip install ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/bin/pip install jupyterlab==3.6 ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/envs/py310/bin/python -m ipykernel install --user --name py310 --display-name py310
RUN echo "配置pip源" \
	&& /opt/conda/bin/pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple/ \
	&& /opt/conda/bin/pip config set global.trusted-host pypi.tuna.tsinghua.edu.cn
RUN echo "配置jupyter" \
    && /opt/conda/bin/jupyter lab --generate-config \
	&& echo "c.ServerApp.allow_origin = '*'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo "c.ServerApp.ip = '0.0.0.0'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo '{"IdentityProvider": {"hashed_password": "argon2:$argon2id$v=19$m=10240,t=10,p=8$I5lrjVBORbgeyAYwXUcAWA$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE"}}' > /root/.jupyter/jupyter_server_config.json \
	&& echo "c.ServerApp.password = 'argon2:\$argon2id\$v=19\$m=10240,t=10,p=8\$I5lrjVBORbgeyAYwXUcAWA\$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE'">> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "c.ServerApp.notebook_dir = '/home'" >> /root/.jupyter/jupyter_lab_config.py \
CMD ["/opt/conda/bin/conda", "run", "jupyter", "lab", "--ip='0.0.0.0'", "--port=8888", "--no-browse", "--allow-root"]
RUN echo "docker 构建完毕 运行命令: docker run --gpus all -p<port>:8888 -v<work_path>:/home --name <name> -d jupyter:latest"



In [2]:
# %load Dockerfile
# 设置基础镜像
# 几个变动  magic_lib.py 可以更新 随即替换文件即可
#         本地源地址更换 替换或更新 pip310_install  和pip_install 文件
#         更新jupyter 启动项  ~/.ipython/profile_default/startup/ 去对应文件夹下查看
#         机器变更 只需要调整  registerd.json host_config.json 的内容就行
#         账号密码管理 暂无
FROM ubuntu:20.04
COPY packages /packages
COPY packages/pip_install.sh /usr/local/bin/pip_install
COPY packages/pip310_install.sh /usr/local/bin/pip310_install
COPY packages/pip310.sh /usr/local/bin/pip310
COPY packages/pipbase.sh /usr/local/bin/pipbase
COPY packages/magic_lib.py /root/.ipython/profile_default/startup/magic_lib.py
SHELL ["/bin/bash", "-c"]
RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install libglib2.0-0 -y \
	&& apt-get install -y wget vim gcc rsync sshpass git libgl1-mesa-glx \
    && chmod +x /usr/local/bin/* /packages/*
RUN echo "下载并安装miniconda" \
	&& bash /packages/download_miniconda.sh \
    && bash miniconda.sh -b -p /opt/conda \
    && rm -rf /packages/download_miniconda.sh \
    && rm -rf miniconda.sh \
	&& ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh
RUN echo "启动conda 修改配置" \
	&& echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && source /root/.bashrc \
	&& source /opt/conda/bin/activate && echo "source /opt/conda/bin/activate" >> ~/.bashrc \
	&& conda create -n py310 python=3.10 -y \
	&& /opt/conda/envs/py310/bin/pip install ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/bin/pip install jupyterlab==3.6 ipykernel==6.28.0 paramiko==3.4.0 scp==0.14.5 dill==0.3.7 fire==0.5.0 pip2pi==0.8.2 \
    && /opt/conda/envs/py310/bin/python -m ipykernel install --user --name py310 --display-name py310
RUN echo "配置pip源" \
	&& /opt/conda/bin/pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple/ \
	&& /opt/conda/bin/pip config set global.trusted-host pypi.tuna.tsinghua.edu.cn
RUN echo "配置jupyter" \
    && /opt/conda/bin/jupyter lab --generate-config \
	&& echo "c.ServerApp.allow_origin = '*'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo "c.ServerApp.ip = '0.0.0.0'" >> /root/.jupyter/jupyter_lab_config.py \
	&& echo '{"IdentityProvider": {"hashed_password": "argon2:$argon2id$v=19$m=10240,t=10,p=8$I5lrjVBORbgeyAYwXUcAWA$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE"}}' > /root/.jupyter/jupyter_server_config.json \
	&& echo "c.ServerApp.password = 'argon2:\$argon2id\$v=19\$m=10240,t=10,p=8\$I5lrjVBORbgeyAYwXUcAWA\$QnIiUIm793Xf6a7Sg3lB23ropGJQ/dcqspxztKsRQqE'">> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "c.ServerApp.notebook_dir = '/home'" >> /root/.jupyter/jupyter_lab_config.py \
 	&& echo "import os;os.chdir('/Users/zhaoxuefeng/GitHub/')" >> /root/.ipython/profile_default/startup/init.py # 添加jupyter启动项
CMD ["/opt/conda/bin/conda", "run", "jupyter", "lab", "--ip='0.0.0.0'", "--port=8888", "--no-browse", "--allow-root"]
RUN echo "docker 构建完毕 运行命令: docker run --gpus all -p<port>:8888 -v<work_path>:/home --name <name> -d jupyter:latest"



In [ ]:
pip2pi qe@192.168.8.125:/qe/data/data/pypi/package $1

In [ ]:
(/opt/conda/bin/pip install $1 --no-cache-dir -i http://192.168.8.125:8100/simple/ --trusted-host 192.168.8.125
) || (echo "本地源缺少该包 使用清华源进行下载";
      /opt/conda/bin/pip install $1 --no-cache-dir -i https://pypi.tuna.tsinghua.edu.cn/simple/;
      echo "同步到本地源";
      sshpass -p qe1234 pip2pi qe@192.168.8.125:/qe/data/data/pypi/package $1;)


In [ ]:
import paramiko
from scp import SCPClient
import fire
import os

# 远程服务器的IP地址、用户名和密码
import json
def load_json(file_path):
    """
    加载JSON文件
    file_path：JSON文件的路径
    返回：JSON文件中的内容，以字典形式返回
    """
    with open(file_path, 'r') as f:
        v = json.load(f)
    return v
host_struct = load_json('/packages/host_config.json') #8840

def sync_pip(package = 'rdt',pip_type='pipbase'):
    if pip_type == 'pipbase':
        pips = 'pip_install'
    elif pip_type == 'pip310':
        pips = 'pip310_install'

    if os.system(f'{pips} {package}'):
        return 'error'
    # 这里更改为支持多机,每台机器上支持多容器的同步模式
    # 重新定义数据结构

    def sync_lower_node(host,username,password,container,package):
        print('开始同步到从机\n')
        # 创建SSH客户端并连接到服务器
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(host, username=username, password=password)
        # 执行远程Python脚本的命令
        command = f'docker exec {container} /bin/bash -c "{pips} {package}"'

        # 通过SSH连接发送命令
        stdin, stdout, stderr = ssh_client.exec_command(command)

        # 获取命令执行的输出和错误
        output = stdout.read().decode()
        errors = stderr.read().decode()

        # 关闭SSH连接
        ssh_client.close()
        # 打印
        print('从机同步完毕\n')
        if errors:
            return errors
        else:
            print(output)

    for node in host_struct:
        for container in node['containers']:
            sync_lower_node(host = node['host'],username = node['username'],password =node['password'],container=container,package=package)



if __name__ == '__main__':
    fire.Fire(sync_pip)


In [ ]:
#!/bin/bash
# 检查参数数量
if [ "$#" -lt 2 ]; then
    echo "Usage: pip310 install [--quick] <filename>"
    exit 1
fi

# 获取操作类型和文件名
operation=$1
quick_mode="false"
filename=""

# 解析参数
for arg in "$@"
do
    case $arg in
        --quick)
            quick_mode="true"
            shift # 移除处理过的参数
            ;;
        *)
            filename=$arg
            ;;
    esac
done

#host = os.environ.get('computer_host','192.168.8.125_126')
# username = os.environ.get('computer_username','qe')
# password = os.environ.get('computer_password','qe1234')
# temp_path = os.environ.get('computer_temp',os.path.join(os.getcwd(),'.computer_temp.py'))
# container = os.environ.get('computer_container',None)
# computer_slave_path = os.environ.get('computer_slave_path','/qe/data/project/project_zxf')

# 根据操作类型和是否快速模式执行相应的命令
if [ "$operation" == "install" ]; then
    if [ "$quick_mode" == "true" ]; then
        echo "Inserting $filename in quick mode"
        # 这里可以放置快速插入的代码
    else
        #echo "Inserting $filename in normal mode"
        # 这里可以放置普通插入的代码
        python /packages/syncpip.py -package $filename -pip_type pip310
        
    fi
elif [ "$operation" == "uninstall" ]; then
    if [ "$quick_mode" == "true" ]; then
        echo "Inserting ssss $filename in quick mode"
        # 这里可以放置快速插入的代码
    else
        echo "Inserting ssss $filename in normal mode"
        # 这里可以放置普通插入的代码
    fi
else
    echo "Unsupported operation: $operation"
    exit 1
fi



In [ ]:
from IPython.core.magic import register_cell_magic
import paramiko
from scp import SCPClient
import os
import sys
import functools
import dill
import IPython

import json
def load_json(file_path):
    """
    加载JSON文件
    file_path：JSON文件的路径
    返回：JSON文件中的内容，以字典形式返回
    """
    with open(file_path, 'r') as f:
        v = json.load(f)
    return v
__registerd_nodes = load_json('/packages/registerd.json') #8840




__cells_temprate = """
import dill
with open('{}', 'rb') as f:
    dicts = dill.load(f)
    locals().update(dicts)
    del f
#print(dicts)
{}
params_dict = dict(locals())
params_dict.pop('dicts')

#返回变量
with open('{}', 'wb') as f:
    dill.dump(params_dict,f)
        """

def __check_line(line):
    if __registerd_nodes.get(line,None):
        return __registerd_nodes[line]
    else:
        raise Exception(f'已注册的有&& {list(__registerd_nodes.keys())} &&')
            
def __build(host):
    # 创建SSH客户端并连接到服务器
    #host = os.environ.get('computer_host','192.168.8.125_126')
    username = os.environ.get('computer_username','qe')
    password = os.environ.get('computer_password','qe1234')
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(host, username=username, password=password)
    return ssh_client


def __runs(ssh_client,command):
    # 通过SSH连接发送命令
    stdin, stdout, stderr = ssh_client.exec_command(command)
    # 获取命令执行的输出和错误
    output = stdout.read().decode()
    errors = stderr.read().decode()
    return output,errors


def __save_params(temp_params_path):
    # 假设您有一个名为fib(n)的函数，用于计算斐波那契数列的第n项
    ip = IPython.get_ipython()
    params = {k: v for k, v in ip.user_ns.items() if
              (not k.startswith('_') and k not in ['In', 'Out', 'exit', 'quit', 'open', 'get_ipython',
                                                  'register_cell_magic','paramiko','SCPClient','sys',
                                                  'dill','IPython','__build','__runs','ssh_computer',
                                                   '__check_line','qenode','qenodeplus','get_params','tknode',
                                                   'load_ipython_extension','os','functools'
                                                  ])}
    with open(temp_params_path, 'wb') as f:
        dill.dump(params, f)
def __load_params(local_params):
    with open(local_params, 'rb') as f:
        dicts = dill.load(f)
        ip = IPython.get_ipython()
        ip.user_ns.update(dicts)
@register_cell_magic
def node(line,cell):
    """
    node_local_file : 环境变量 指定主机的缓存文件位置 
    slave_file : 环境变量 指定从机的缓存文件位置 
    slave_docker_file:
    """    
    local_file = os.environ.get('node_local_file',os.path.join(os.getcwd(),'.computer_temp.py'))
    slave_file = os.environ.get('slave_file',os.path.join(os.getcwd(),'/qe/data/data/.temp.py'))
    slave_docker_file = '/root/temp.py' #从机docker文件
    local_params = os.path.join(local_file.rsplit('/',1)[0],'.params_path') #主机变量位置
    slave_params = os.path.join(slave_file.rsplit('/',1)[0],'.params_path') #从机变量位置
    slave_docker_params = '/root/temp.params_path' #从机docker变量
    # 检查参数
    infos = __check_line(line)

    #保存参数
    __save_params(local_params)
    
    slave_docker_params_ = slave_docker_params+'_'
    slave_params_ = slave_params+'_'
    local_params_ = local_params+'_'
    #保存代码
    cells = __cells_temprate.format(slave_docker_params,cell,slave_docker_params_)
    with open(local_file, 'w') as f:
        f.write(cells)
    
    #传输
    ssh_client = __build(infos['host'])
    with SCPClient(ssh_client.get_transport()) as scp:
        scp.put(local_file, slave_file)
        scp.put(local_params,slave_params)
            
    
    __runs(ssh_client,f'docker cp {slave_file} {infos["container"]}:{slave_docker_file}')
    __runs(ssh_client,f'docker cp {slave_params} {infos["container"]}:{slave_docker_params}')
    
    kernel_name = os.path.basename(sys.executable.replace("/bin/python",""))
    if kernel_name == 'py310':
        output,errors = __runs(ssh_client,f'docker exec {infos["container"]} /bin/bash -c "/opt/conda/envs/py310/bin/python /root/temp.py"')
    elif kernel_name == 'conda':
        output,errors = __runs(ssh_client,f'docker exec {infos["container"]} /bin/bash -c "/opt/conda/bin/python /root/temp.py"')
    #拉取docker 文件 dockerfiles cp mycontainer:/opt/test.txt /home/user
    
    __runs(ssh_client,f'docker cp {infos["container"]}:{slave_docker_params_} {slave_params_}')
    
    with SCPClient(ssh_client.get_transport()) as scp:
        scp.get(slave_params_, local_params_)
        
    __load_params(local_params_)
    # 关闭SSH连接
    ssh_client.close()

    if errors:
        raise Exception(errors)
    else:
        print(output)



@register_cell_magic
def nodeold(line,cell):
    """
    node_local_file : 环境变量 指定主机的缓存文件位置 
    slave_file : 环境变量 指定从机的缓存文件位置 
    slave_docker_file:
    """    
    local_file = os.environ.get('node_local_file',os.path.join(os.getcwd(),'.computer_temp.py'))
    slave_file = os.environ.get('slave_file',os.path.join(os.getcwd(),'/qe/data/data/.temp.py'))
    slave_docker_file = '/root/temp.py' #从机docker文件
    infos = __check_line(line)
    
    with open(local_file,'w') as f:
        f.write(cell)
        
    ssh_client = __build(infos['host'])
    with SCPClient(ssh_client.get_transport()) as scp:
        scp.put(local_file, slave_file)

    __runs(ssh_client,f'docker cp {slave_file} {infos["container"]}:{slave_docker_file}')
    kernel_name = os.path.basename(sys.executable.replace("/bin/python",""))
    if kernel_name == 'py310':
        output,errors = __runs(ssh_client,f'docker exec {infos["container"]} /bin/bash -c "/opt/conda/envs/py310/bin/python /root/temp.py"')
    elif kernel_name == 'conda':
        output,errors = __runs(ssh_client,f'docker exec {infos["container"]} /bin/bash -c "/opt/conda/bin/python /root/temp.py"')
    # 关闭SSH连接
    ssh_client.close()

    if errors:
        raise Exception(errors)
    else:
        print(output)


@register_cell_magic
def test(line,cell):
    print(line,'line',type(line))
    print(cell,'cell',type(cell))

# 为了能够在IPython中使用这个魔法命令，需要加载这个扩展
def load_ipython_extension(ipython):
    ipython.register_magic_function(test, 'cell', 'test')
    ipython.register_magic_function(nodeold, 'cell', 'nodeold')
    ipython.register_magic_function(node, 'cell', 'tknode')
    